In [374]:
import pandas as pd

## Extracting Temperature and Tariff data

In [375]:
# Getting the data from Arduino CLoud that was automatically collected
d1 = pd.read_csv('iot_project-electricityRate.csv') # Electricity Tariff
d2 = pd.read_csv('iot_project-homeTemp.csv') # Internal Room Temp
d3 = pd.read_csv('iot_project-outdoorTemp.csv') # Outdoor Weather Temp

In [376]:
d3.shape

(8298, 2)

In [377]:
d1.head()

,time,value
0,2024-12-05T00:00:41.181931534Z,18.480000
1,2024-12-05T00:01:53.911248607Z,19.467001
2,2024-12-05T00:03:07.0999956Z,19.467001
3,2024-12-05T00:04:20.117557107Z,19.467001
4,2024-12-05T00:05:33.028128013Z,19.467001


In [378]:
def standardize_timestamps(df):
    df = df.copy()
    
    # Convert the time to datetime if it's not already
    df['time'] = pd.to_datetime(df['time'])
    
    # Group by rounded timestamps and take the first occurrence to remove duplicates
    df = df.groupby('time').first().reset_index()
    
    # Set time as index and sort
    df.set_index('time', inplace=True)
    df.sort_index(inplace=True)
    
    return df

In [379]:
# Round timestamps to the nearest 30 minutes for consistent merging
def round_timestamps_one(df):
    df.index = df.index.round('1min')
    return df    

In [380]:
# Round timestamps to the nearest 30 minutes for consistent merging
def round_timestamps(df):
    df.index = df.index.round('30min')
    return df    

In [381]:
d1.head()

,time,value
0,2024-12-05T00:00:41.181931534Z,18.480000
1,2024-12-05T00:01:53.911248607Z,19.467001
2,2024-12-05T00:03:07.0999956Z,19.467001
3,2024-12-05T00:04:20.117557107Z,19.467001
4,2024-12-05T00:05:33.028128013Z,19.467001


In [382]:
d1 = standardize_timestamps(d1)
d2 = standardize_timestamps(d2)
d3 = standardize_timestamps(d3)

In [383]:
# Merging the data in one df
data = d1.merge(d2, on='time', how='outer')
data = data.merge(d3, on='time', how='outer')

In [384]:
data = data.rename(columns={
    'value_x': 'electricity_rate',
    'value_y': 'indoor_temp',
    'value': 'outdoor_temp'
})

In [385]:
data.shape

(8304, 3)

In [386]:
data.dropna(inplace=True)
data.shape

(8292, 3)

In [387]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8292 entries, 2024-12-05 00:00:41.181931534+00:00 to 2024-12-12 01:04:06.053114536+00:00
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   electricity_rate  8292 non-null   float64
 1   indoor_temp       8292 non-null   float64
 2   outdoor_temp      8292 non-null   float64
dtypes: float64(3)
memory usage: 259.1 KB


In [388]:
data.describe()

,electricity_rate,indoor_temp,outdoor_temp
count,8292.000000,8292.000000,8292.000000
mean,12.214796,18.320964,8.016564
std,6.732691,1.837422,2.113489
min,0.420000,12.250000,5.670000
25%,5.040000,18.062500,6.610000
50%,13.860000,18.937500,7.200000
75%,18.018000,19.187500,8.860000
max,34.996498,20.625000,14.290000


In [389]:
# Importing electricity consumption data
d4 = pd.read_csv('electricity_consuption.csv')

In [390]:
d4.keys()

Index(['Consumption (kwh)', ' Estimated Cost Inc. Tax (p)', ' Start', ' End'], dtype='object')

In [391]:
# Fixing the columns of consumption data
d4[' Start'] = pd.to_datetime(d4[' Start'])
d4.drop(columns=' End', inplace=True)
d4 = d4.rename(columns={
    ' Start': 'time'
})

In [392]:
# Standardizing and rounding the consumption data
data_energy = round_timestamps(standardize_timestamps(d4))

In [393]:
# Data raw will be used for the sleep data
round_timestamps_one(data)

,electricity_rate,indoor_temp,outdoor_temp
time,,,
2024-12-05 00:01:00+00:00,18.480000,20.0000,9.400001
2024-12-05 00:02:00+00:00,19.467001,19.9375,9.400001
2024-12-05 00:03:00+00:00,19.467001,19.9375,9.400001
2024-12-05 00:04:00+00:00,19.467001,19.9375,9.400001
2024-12-05 00:06:00+00:00,19.467001,19.9375,9.400001
...,...,...,...
2024-12-12 00:59:00+00:00,18.270000,18.8750,6.510000
2024-12-12 01:00:00+00:00,18.270000,18.8750,6.510000
2024-12-12 01:02:00+00:00,17.829000,18.8750,6.510000


In [394]:
data_raw = data.copy()

In [395]:
data.head()

,electricity_rate,indoor_temp,outdoor_temp
time,,,
2024-12-05 00:01:00+00:00,18.480000,20.0000,9.400001
2024-12-05 00:02:00+00:00,19.467001,19.9375,9.400001
2024-12-05 00:03:00+00:00,19.467001,19.9375,9.400001
2024-12-05 00:04:00+00:00,19.467001,19.9375,9.400001
2024-12-05 00:06:00+00:00,19.467001,19.9375,9.400001


In [396]:
data.to_csv('temperature_tariff_data.csv')

## Electricity Consumption Data

In [397]:
# Rounding the other dataset
data_grouped = round_timestamps(data)

In [398]:
# Group the other dataset by the rounded timestamps and take the first/mean values
data_grouped = data_grouped.groupby('time').agg({
    'electricity_rate': 'mean',
    'indoor_temp': 'mean',
    'outdoor_temp': 'mean'
}).reset_index()

In [399]:
# Merging the consuption data with the tariff and temperature data
data_energy = data_energy.merge(
    data_grouped, 
    left_on='time', 
    right_on='time', 
    how='left'
)

In [400]:
data_energy.dropna(inplace=True)

In [401]:
data_energy.keys()

Index(['time', 'Consumption (kwh)', ' Estimated Cost Inc. Tax (p)',
       'electricity_rate', 'indoor_temp', 'outdoor_temp'],
      dtype='object')

In [402]:
data_energy = data_energy.rename(columns={
    'Consumption (kwh)': 'consumption',
    ' Estimated Cost Inc. Tax (p)': 'cost',
})

In [403]:
data_energy.keys()

Index(['time', 'consumption', 'cost', 'electricity_rate', 'indoor_temp',
       'outdoor_temp'],
      dtype='object')

In [404]:
data_energy.to_csv('consumption_temperature_data.csv')

## Extracting sleep data

In [487]:
d5 = pd.read_csv('SleepData.csv')

In [488]:
d5.columns

Index(['Date', 'Time REM(hr)', 'Time asleep(hr)', 'Time awake(hr)',
       'Time core(hr)', 'Time deep(hr)'],
      dtype='object')

In [489]:
d5['time'] = d5['Date'].str.split(" - ").str[0]
d5.drop(columns='Date', inplace=True)

In [490]:
d5.shape

(220, 6)

In [491]:
sleep_data = standardize_timestamps(d5)

In [492]:
sleep_data = round_timestamps_one(sleep_data)

In [493]:
print(sleep_data.head())

                     Time REM(hr)  Time asleep(hr)  Time awake(hr)  \
time                                                                 
2024-12-01 02:05:00           NaN              NaN             NaN   
2024-12-01 02:25:00           NaN              NaN             NaN   
2024-12-01 03:11:00           NaN              NaN             NaN   
2024-12-01 04:07:00         0.092              NaN             NaN   
2024-12-01 04:13:00           NaN              NaN             NaN   

                     Time core(hr)  Time deep(hr)  
time                                               
2024-12-01 02:05:00          0.325            NaN  
2024-12-01 02:25:00            NaN          0.767  
2024-12-01 03:11:00          0.942            NaN  
2024-12-01 04:07:00            NaN            NaN  
2024-12-01 04:13:00          0.192            NaN  


In [494]:
def fill_sleep_type(row):
    for col in row.index:
        if not pd.isna(row[col]):
            return col.replace("Time ", "").replace("(hr)", "").strip()

In [495]:
sleep_data['sleep_type'] = sleep_data.apply(fill_sleep_type, axis=1)

In [496]:
sleep_data.keys()

Index(['Time REM(hr)', 'Time asleep(hr)', 'Time awake(hr)', 'Time core(hr)',
       'Time deep(hr)', 'sleep_type'],
      dtype='object')

In [497]:
sleep_data.head()

,Time REM(hr),Time asleep(hr),Time awake(hr),Time core(hr),Time deep(hr),sleep_type
time,,,,,,
2024-12-01 02:05:00,NaN,NaN,NaN,0.325,NaN,core
2024-12-01 02:25:00,NaN,NaN,NaN,NaN,0.767,deep
2024-12-01 03:11:00,NaN,NaN,NaN,0.942,NaN,core
2024-12-01 04:07:00,0.092,NaN,NaN,NaN,NaN,REM
2024-12-01 04:13:00,NaN,NaN,NaN,0.192,NaN,core


In [498]:
sleep_data.drop(columns=['Time REM(hr)','Time asleep(hr)','Time awake(hr)', 'Time core(hr)',
       'Time deep(hr)'], inplace=True)

In [499]:
data_raw.shape

(8292, 3)

In [500]:
sleep_data.index = sleep_data.index +pd.to_timedelta(4, unit='d')

In [482]:
print(data_raw.head())

                           electricity_rate  indoor_temp  outdoor_temp
time                                                                  
2024-12-05 00:01:00+00:00         18.480000      20.0000      9.400001
2024-12-05 00:02:00+00:00         19.467001      19.9375      9.400001
2024-12-05 00:03:00+00:00         19.467001      19.9375      9.400001
2024-12-05 00:04:00+00:00         19.467001      19.9375      9.400001
2024-12-05 00:06:00+00:00         19.467001      19.9375      9.400001


In [484]:
print(sleep_data.head())

                    sleep_type
time                          
2024-12-05 02:05:00       core
2024-12-05 02:25:00       deep
2024-12-05 03:11:00       core
2024-12-05 04:07:00        REM
2024-12-05 04:13:00       core


In [502]:
sleep_data.index = sleep_data.index.tz_localize("UTC")

In [504]:
sleep_data = data_raw.merge(sleep_data, left_index=True, right_index=True, how="outer")

In [507]:
sleep_data = sleep_data.dropna()

In [508]:
sleep_data.head()

,electricity_rate,indoor_temp,outdoor_temp,sleep_type
time,,,,
2024-12-05 02:05:00+00:00,17.871000,19.8125,10.02,core
2024-12-05 02:25:00+00:00,18.059999,19.8125,10.19,deep
2024-12-05 04:07:00+00:00,17.871000,19.8750,10.63,REM
2024-12-05 04:13:00+00:00,18.059999,19.8750,10.62,core
2024-12-05 04:24:00+00:00,18.059999,19.9375,10.62,REM


In [509]:
sleep_data.to_csv('temperature_sleep_data.csv')